# Implementation of FaceForensics++: Learning to Detect Manipulated Facial Images


In [4]:
import keras
import tensorflow as tf
from keras import layers,Model
from keras import models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.layers import Dropout,InputLayer,Flatten,Dense,BatchNormalization,MaxPooling2D,Conv2D,Input,Concatenate,LeakyReLU
from keras import optimizers
#from tensorflow.keras.utils import to_categorical,normalize
from keras.applications.xception import Xception,preprocess_input
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint,TensorBoard

import joblib

import warnings
import keras.backend as K

warnings.filterwarnings('ignore')

In [5]:
train_dataset = image_dataset_from_directory('./data/train', 
                                                labels="inferred", 
                                                label_mode="int",
                                                color_mode="rgb",
                                                image_size=(299, 299)
                                                )

val_dataset = image_dataset_from_directory('./data/val', 
                                                labels="inferred", 
                                                label_mode="int",
                                                color_mode="rgb",
                                                image_size=(299, 299)
                                                )

test_dataset = image_dataset_from_directory('./data/test', 
                                                labels="inferred", 
                                                label_mode="int",
                                                color_mode="rgb",
                                                image_size=(299, 299)
                                                )

Found 11448 files belonging to 2 classes.
Found 1257 files belonging to 2 classes.
Found 3103 files belonging to 2 classes.


In [6]:
class_names = train_dataset.class_names
print(class_names)

['fake', 'real']


In [7]:
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 299, 299, 3)
(32,)


In [8]:
Xception_initial=Xception(include_top=False,
                 weights='imagenet',
                 input_shape=(299,299,3),pooling ='avg',
                 )

In [9]:
for layer in Xception_initial.layers:
    layer.trainable = True

x = Xception_initial.output

predicted = Dense(1, activation="sigmoid")(x) # binary activation output

model_pretrain = Model(inputs = Xception_initial.input, outputs = predicted)

model_pretrain.compile(loss=keras.losses.binary_crossentropy,
          optimizer=tf.keras.optimizers.legacy.Adam(lr = 0.0002),
          metrics=['accuracy'])

pretraining_Xception =model_pretrain.fit(train_dataset,
                                        verbose=1,
                                        batch_size=32, 
                                        epochs=3
                                        )

Epoch 1/3


358/358 [==============================] - 1874s 5s/step - loss: 0.0321 - accuracy: 0.9863
Epoch 2/3
358/358 [==============================] - 1828s 5s/step - loss: 5.3494e-04 - accuracy: 0.9999
Epoch 3/3
358/358 [==============================] - 1832s 5s/step - loss: 0.0015 - accuracy: 0.9997


In [10]:
model_pretrain.layers.pop()

In [11]:
tensorboard=TensorBoard(log_dir='./logs', histogram_freq=3)

In [12]:
for layer in model_pretrain.layers:
    layer.trainable = False
    
x = model_pretrain.output

predicted = Dense(1, activation="sigmoid")(x) 

model_finetune1 = Model(inputs = model_pretrain.input, outputs = predicted)

model_finetune1.compile(loss=keras.losses.binary_crossentropy,
          optimizer=tf.keras.optimizers.Adam(lr = 0.0002),
          metrics=['accuracy'])


finetuning1_XceptionNet=model_finetune1.fit(train_dataset,
                                            verbose=1,
                                            batch_size=16,
                                            epochs=15,
                                            validation_data=val_dataset,
                                            callbacks=[tensorboard]
                                             )

Epoch 1/15
358/358 [==============================] - 492s 1s/step - loss: 0.6842 - accuracy: 0.5644 - val_loss: 0.6433 - val_accuracy: 1.0000
Epoch 2/15
358/358 [==============================] - 482s 1s/step - loss: 0.6073 - accuracy: 1.0000 - val_loss: 0.5710 - val_accuracy: 1.0000
Epoch 3/15
358/358 [==============================] - 482s 1s/step - loss: 0.5392 - accuracy: 1.0000 - val_loss: 0.5068 - val_accuracy: 1.0000
Epoch 4/15
358/358 [==============================] - 476s 1s/step - loss: 0.4786 - accuracy: 1.0000 - val_loss: 0.4497 - val_accuracy: 1.0000
Epoch 5/15
358/358 [==============================] - 508s 1s/step - loss: 0.4248 - accuracy: 1.0000 - val_loss: 0.3992 - val_accuracy: 1.0000
Epoch 6/15
358/358 [==============================] - 498s 1s/step - loss: 0.3772 - accuracy: 1.0000 - val_loss: 0.3546 - val_accuracy: 1.0000
Epoch 7/15
358/358 [==============================] - 506s 1s/step - loss: 0.3351 - accuracy: 1.0000 - val_loss: 0.3151 - val_accuracy: 1.0000

In [13]:
model_finetune1.evaluate(train_dataset)

358/358 [==============================] - 426s 1s/step - loss: 0.1254 - accuracy: 1.0000


[0.12538781762123108, 1.0]

In [14]:
model_finetune1.evaluate(val_dataset)

40/40 [==============================] - 47s 1s/step - loss: 0.1252 - accuracy: 1.0000


[0.1252121925354004, 1.0]

In [15]:
model_finetune1.evaluate(test_dataset)

97/97 [==============================] - 115s 1s/step - loss: 0.1253 - accuracy: 1.0000


[0.12530866265296936, 1.0]

In [16]:
model_finetune1.save("modelload.hdf5")